In [158]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from hts import HTSRegressor
import hts.functions
from hts.hierarchy import HierarchyTree
from sklearn.pipeline import Pipeline
import json
%matplotlib inline

In [88]:
zri_df_og = pd.read_csv('../../data/HIER_zri_multifamily_v2.csv', dtype={'zip': str})

In [89]:
zri_df = zri_df_og.drop(['SizeRank', 'Metro'], axis=1)

In [90]:
zri_df['State_County'] = zri_df['State'] +\
                                        "_" + zri_df['CountyName']
zri_df['State_County_City'] = zri_df['State'] +\
                                        "_" + zri_df['CountyName'] +\
                                        "_" + zri_df['City']
zri_df['State_County_City_Zip'] = zri_df['State'] +\
                                        "_" + zri_df['CountyName'] +\
                                        "_" + zri_df['City'] +\
                                        "_" + zri_df['zip'] 
zri_df

,zip,City,State,CountyName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,...,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,State_County,State_County_City,State_County_City_Zip
0,10025,New York,NY,New York County,3333.0,3338.0,3340.0,3323.0,3316.0,3318.0,...,3622.0,3664.0,3698.0,3704.0,3692.0,3715.0,3676.0,NY_New York County,NY_New York County_New York,NY_New York County_New York_10025
1,60657,Chicago,IL,Cook County,1647.0,1645.0,1635.0,1650.0,1660.0,1665.0,...,1787.0,1801.0,1806.0,1789.0,1761.0,1747.0,1731.0,IL_Cook County,IL_Cook County_Chicago,IL_Cook County_Chicago_60657
2,10023,New York,NY,New York County,3135.0,3138.0,3136.0,3131.0,3118.0,3122.0,...,3491.0,3510.0,3521.0,3574.0,3608.0,3561.0,NaN,NY_New York County,NY_New York County_New York,NY_New York County_New York_10023
3,77494,Katy,TX,Harris County,NaN,NaN,1081.0,1093.0,1113.0,1118.0,...,1251.0,1246.0,1252.0,1280.0,1310.0,1294.0,1286.0,TX_Harris County,TX_Harris County_Katy,TX_Harris County_Katy_77494
4,60614,Chicago,IL,Cook County,1756.0,1759.0,1754.0,1774.0,1780.0,1778.0,...,1950.0,1953.0,1952.0,1921.0,1958.0,NaN,1916.0,IL_Cook County,IL_Cook County_Chicago,IL_Cook County_Chicago_60614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,04937,Fairfield,ME,Somerset County,794.0,793.0,790.0,792.0,793.0,774.0,...,1105.0,1113.0,1113.0,1146.0,NaN,NaN,NaN,ME_Somerset County,ME_Somerset County_Fairfield,ME_Somerset County_Fairfield_04937
1298,60606,Chicago,IL,Cook County,2004.0,2001.0,2016.0,2041.0,2066.0,2083.0,...,2183.0,2206.0,2228.0,2217.0,2230.0,2200.0,2192.0,IL_Cook County,IL_Cook County_Chicago,IL_Cook County_Chicago_60606
1299,12307,Schenectady,NY,Schenectady County,NaN,1044.0,NaN,1028.0,NaN,NaN,...,974.0,978.0,978.0,NaN,NaN,1039.0,1047.0,NY_Schenectady County,NY_Schenectady County_Schenectady,NY_Schenectady County_Schenectady_12307
1300,18232,Lansford,PA,Carbon County,775.0,778.0,780.0,783.0,779.0,779.0,...,762.0,758.0,758.0,NaN,NaN,NaN,NaN,PA_Carbon County,PA_Carbon County_Lansford,PA_Carbon County_Lansford_18232


In [94]:
# list(zri_df.columns)

['zip',
 'City',
 'State',
 'CountyName',
 '2014-01',
 '2014-02',
 '2014-03',
 '2014-04',
 '2014-05',
 '2014-06',
 '2014-07',
 '2014-08',
 '2014-09',
 '2014-10',
 '2014-11',
 '2014-12',
 '2015-01',
 '2015-02',
 '2015-03',
 '2015-04',
 '2015-05',
 '2015-06',
 '2015-07',
 '2015-08',
 '2015-09',
 '2015-10',
 '2015-11',
 '2015-12',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12',
 '2020-01',
 'State_County',
 'State_County_City',
 'State_County_City_Zip']

In [164]:
zri_by_total = pd.DataFrame(zri_df[['2014-01',
 '2014-02',
 '2014-03',
 '2014-04',
 '2014-05',
 '2014-06',
 '2014-07',
 '2014-08',
 '2014-09',
 '2014-10',
 '2014-11',
 '2014-12',
 '2015-01',
 '2015-02',
 '2015-03',
 '2015-04',
 '2015-05',
 '2015-06',
 '2015-07',
 '2015-08',
 '2015-09',
 '2015-10',
 '2015-11',
 '2015-12',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12',
 '2020-01']].mean()).T
zri_by_total['Total']='total'
zri_by_total = zri_by_total.set_index('Total')

In [78]:
zri_by_state = zri_df.groupby(["State"]).mean()

In [79]:
zri_by_county = zri_df.groupby(["State_County"]).mean()

In [80]:
zri_by_city = zri_df.groupby(["State_County_City"]).mean()

In [128]:
zri_by_city

,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,2014-10,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
State_County_City,,,,,,,,,,,,,,,,,,,,,
AK_Anchorage Borough_Anchorage,1423.5,1387.0,1381.0,1369.0,1348.666667,1336.0,1344.333333,1364.666667,1384.666667,1404.333333,...,1298.666667,1303.333333,1306.666667,1308.333333,1310.333333,1315.0,1333.333333,1340.0,1333.333333,1335.666667
AK_Matanuska Susitna Borough_Wasilla,NaN,1192.0,1210.0,1216.0,1206.000000,1201.0,1225.000000,1220.000000,1217.000000,1234.000000,...,1307.000000,1296.000000,1294.000000,1291.000000,1277.000000,1271.0,1299.000000,NaN,1224.000000,NaN
AR_Benton County_Bentonville,NaN,NaN,799.0,809.0,820.000000,831.0,834.000000,837.000000,832.000000,827.000000,...,917.000000,917.000000,921.000000,920.000000,919.000000,914.0,914.000000,915.0,916.000000,929.000000
AR_Pulaski County_Little Rock,NaN,667.0,661.0,664.0,665.000000,669.0,667.000000,666.000000,665.000000,669.000000,...,720.000000,713.000000,711.000000,711.000000,714.000000,721.0,735.000000,728.0,726.000000,727.000000
AR_Washington County_Springdale,NaN,NaN,844.0,850.0,850.000000,855.0,857.000000,865.000000,859.000000,843.000000,...,928.000000,935.000000,936.000000,931.000000,931.000000,934.0,959.000000,954.0,962.000000,954.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WI_Milwaukee County_Shorewood,1058.0,1053.0,1063.0,1065.0,1063.000000,1070.0,1078.000000,1089.000000,1103.000000,1110.000000,...,1205.000000,1197.000000,1200.000000,1199.000000,1191.000000,1178.0,1181.000000,1180.0,1185.000000,NaN
WI_Racine County_Mount Pleasant,NaN,NaN,890.0,876.0,862.000000,849.0,862.000000,886.000000,905.000000,931.000000,...,845.000000,825.000000,806.000000,790.000000,790.000000,801.0,NaN,835.0,838.000000,817.000000
WI_Winnebago County_Oshkosh,NaN,NaN,NaN,653.0,642.000000,637.0,634.000000,641.000000,651.000000,NaN,...,688.000000,687.000000,NaN,695.000000,702.000000,700.0,NaN,742.0,742.000000,748.000000


In [165]:
df_dict_total={}
df_dict_total['total']=list(zri_df['State'].unique())

In [166]:
df_dict_states={}
for state in df_dict_total['total']:
    df_dict_states[state]=list(zri_df[zri_df['State']==state]['State_County'].unique())

In [167]:
df_dict_county={}
for state in df_dict_total['total']:
    for county in df_dict_states[state]:
        df_dict_county[county]=list(zri_df[zri_df['State_County']==county]['State_County_City'].unique())

In [168]:
df_dict_city={}
for state in df_dict_total['total']:
    for county in df_dict_states[state]:
        for city in df_dict_county[county]:
            df_dict_city[city]=list(zri_df[zri_df['State_County_City']==city]['State_County_City_Zip'].unique())

In [169]:
hierarchy = {**df_dict_total, **df_dict_states, **df_dict_county, **df_dict_city}


with open('../../data/processed/hierarchy.json', 'w') as j:
    json.dump(hierarchy, j)

In [170]:
zri_by_zip = zri_df.set_index('State_County_City_Zip')\
.drop(['zip', 'City', 'State', 'CountyName',
      'State_County', 'State_County_City'], axis=1)

In [171]:
all_zri = [zri_by_total, zri_by_state, zri_by_county, zri_by_city, zri_by_zip]
full_zri = pd.concat(all_zri).T
# full_zri

In [172]:
full_zri

,total,AK,AR,AZ,CA,CO,CT,DC,FL,GA,...,NY_Erie County_Buffalo_14208,CT_Fairfield County_Stamford_06906,MA_Suffolk County_Boston_02210,CT_Fairfield County_Norwalk_06855,CT_Fairfield County_Bridgeport_06607,ME_Somerset County_Fairfield_04937,IL_Cook County_Chicago_60606,NY_Schenectady County_Schenectady_12307,PA_Carbon County_Lansford_18232,TX_Dallas County_Dallas_75202
2014-01,1450.300125,1423.500000,NaN,854.600000,1744.284404,1026.500000,1211.277778,2057.727273,1300.852459,960.250000,...,NaN,NaN,NaN,NaN,NaN,794.0,2004.0,NaN,775.0,1297.0
2014-02,1401.843870,1338.250000,667.000000,798.642857,1752.892857,1131.153846,1214.928571,2025.769231,1286.073684,1044.466667,...,646.0,NaN,2668.0,1765.0,1190.0,793.0,2001.0,1044.0,778.0,1284.0
2014-03,1401.135822,1338.250000,768.000000,851.523810,1842.027027,1150.280000,1224.529412,2017.153846,1265.612613,1030.666667,...,658.0,1863.0,2704.0,1760.0,1197.0,790.0,2016.0,NaN,780.0,1295.0
2014-04,1406.572332,1330.750000,774.333333,858.047619,1849.185185,1191.115385,1223.686275,2001.076923,1263.392857,1045.040000,...,667.0,1848.0,2729.0,1731.0,1201.0,792.0,2041.0,1028.0,783.0,1297.0
2014-05,1407.971631,1313.000000,778.333333,863.380952,1853.015385,1209.600000,1223.117647,1987.076923,1259.754545,1059.360000,...,670.0,1847.0,2767.0,1714.0,1201.0,793.0,2066.0,NaN,779.0,1296.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09,1690.725155,1304.000000,856.333333,1183.761905,2365.415385,1563.153846,1337.431373,2187.538462,1498.125000,1401.333333,...,829.0,1958.0,3085.0,1905.0,1342.0,1113.0,2228.0,978.0,758.0,1729.0
2019-10,1706.941075,1324.750000,869.333333,1201.300000,2358.239130,1570.307692,1344.755102,2198.307692,1515.540000,1445.869565,...,844.0,1918.0,3049.0,1906.0,NaN,1146.0,2217.0,NaN,NaN,1697.0
2019-11,1685.127768,1340.000000,865.666667,1216.400000,2380.910615,1595.440000,1364.851064,2210.846154,1506.485981,1448.760000,...,847.0,1938.0,3054.0,1901.0,NaN,NaN,2230.0,NaN,NaN,1739.0
2019-12,1694.832215,1306.000000,868.000000,1194.133333,2361.595745,1623.250000,1362.673913,2190.818182,1506.766355,1470.875000,...,850.0,1920.0,3019.0,1902.0,1347.0,NaN,2200.0,1039.0,NaN,NaN


In [146]:
tree = HierarchyTree.from_nodes(df_dict, full_zri, root='Total')

In [147]:
print(tree)

- Total
   |- NY
   |  |- NY_New York County
   |  |  - NY_New York County_New York
   |  |     |- NY_New York County_New York_10025
   |  |     |- NY_New York County_New York_10023
   |  |     |- NY_New York County_New York_10002
   |  |     |- NY_New York County_New York_10016
   |  |     |- NY_New York County_New York_10029
   |  |     |- NY_New York County_New York_10009
   |  |     |- NY_New York County_New York_10011
   |  |     |- NY_New York County_New York_10128
   |  |     |- NY_New York County_New York_10019
   |  |     |- NY_New York County_New York_10003
   |  |     |- NY_New York County_New York_10463
   |  |     |- NY_New York County_New York_10024
   |  |     |- NY_New York County_New York_10028
   |  |     |- NY_New York County_New York_10027
   |  |     |- NY_New York County_New York_10021
   |  |     |- NY_New York County_New York_10031
   |  |     |- NY_New York County_New York_10014
   |  |     |- NY_New York County_New York_10036
   |  |     |- NY_New York County_

In [173]:
clf = HTSRegressor(model='prophet', revision_method='OLS', n_jobs=12)
model = clf.fit(full_zri, hierarchy)

Fitting models:   0%|          | 0/60 [00:08<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'fit'

In [ ]:
print(f'Node: {ht.children[0].children[2].children[2].key}, item: ')
ht.children[0].children[2].children[2].item

In [111]:
grouped_sections = zri_df.groupby(["State", "State_County", 
                                   "State_County_City", "State_County_City_Zip"])
edges_hierarchy = list(grouped_sections.groups.keys())
edges_hierarchy[:20]

[('AK',
  'AK_Anchorage Borough',
  'AK_Anchorage Borough_Anchorage',
  'AK_Anchorage Borough_Anchorage_99501'),
 ('AK',
  'AK_Anchorage Borough',
  'AK_Anchorage Borough_Anchorage',
  'AK_Anchorage Borough_Anchorage_99504'),
 ('AK',
  'AK_Anchorage Borough',
  'AK_Anchorage Borough_Anchorage',
  'AK_Anchorage Borough_Anchorage_99508'),
 ('AK',
  'AK_Matanuska Susitna Borough',
  'AK_Matanuska Susitna Borough_Wasilla',
  'AK_Matanuska Susitna Borough_Wasilla_99654'),
 ('AR',
  'AR_Benton County',
  'AR_Benton County_Bentonville',
  'AR_Benton County_Bentonville_72712'),
 ('AR',
  'AR_Pulaski County',
  'AR_Pulaski County_Little Rock',
  'AR_Pulaski County_Little Rock_72205'),
 ('AR',
  'AR_Washington County',
  'AR_Washington County_Springdale',
  'AR_Washington County_Springdale_72764'),
 ('AZ',
  'AZ_Maricopa County',
  'AZ_Maricopa County_Chandler',
  'AZ_Maricopa County_Chandler_85224'),
 ('AZ',
  'AZ_Maricopa County',
  'AZ_Maricopa County_Mesa',
  'AZ_Maricopa County_Mesa_85201')

In [25]:
states = list(zri_df.State.unique())

In [26]:
second_level_nodes = states
root_node = "total"

root_edges = [(root_node, second_level_node) for second_level_node in second_level_nodes]
# root_edges

In [27]:
root_edges += edges_hierarchy

In [20]:
tree = HierarchyTree.from_nodes(root_edges, zri_df, root='total')
print(tree)

KeyError: "None of [Index(['total'], dtype='object')] are in the [columns]"

In [21]:
zri_df

,zip,City,State,Metro,CountyName,year-month,zri
0,1013,Chicopee,MA,Springfield,Hampden County,2014-01,928.0
1,1013,Chicopee,MA,Springfield,Hampden County,2014-02,931.0
2,1013,Chicopee,MA,Springfield,Hampden County,2014-03,934.0
3,1013,Chicopee,MA,Springfield,Hampden County,2014-04,929.0
4,1013,Chicopee,MA,Springfield,Hampden County,2014-05,929.0
...,...,...,...,...,...,...,...
93739,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-08,1277.0
93740,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-09,1271.0
93741,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-10,1299.0
93742,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,2019-11,1261.5


In [17]:
Xn, Yn, Xe, Ye, labels, annot = get_nodes_edges_position(root_edges, root="total")
M = max(Yn)

fig = go.Figure()

fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))

fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=30,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))

axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(title= 'Hierarchical Structure Tree for Australian Tourism Data',
              annotations=annot,
              font_size=10,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=40, r=40, b=85, t=100),
              hovermode='closest',
              plot_bgcolor='rgb(248,248,248)'
              )
fig.show()

NameError: name 'get_nodes_edges_position' is not defined

In [55]:
idx = pd.MultiIndex.from_product([['python', 'cobra'],
                                   [2018, 2019]])
idx


MultiIndex([('python', 2018),
            ('python', 2019),
            ( 'cobra', 2018),
            ( 'cobra', 2019)],
           )

In [56]:

idx.set_names(['kind', 'year'], inplace=True)
idx


MultiIndex([('python', 2018),
            ('python', 2019),
            ( 'cobra', 2018),
            ( 'cobra', 2019)],
           names=['kind', 'year'])

In [57]:
idx.set_names('species', level=0)


MultiIndex([('python', 2018),
            ('python', 2019),
            ( 'cobra', 2018),
            ( 'cobra', 2019)],
           names=['species', 'year'])